In [73]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable 
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchvision import datasets, transforms
# from PIL import Imageimport matplotlib
import matplotlib
import matplotlib.pyplot as plt

In [74]:
BATCH_SIZE=32 #大概需要2G的显存
EPOCHS=5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataset = pd.read_csv('mnist_train.csv')
test_dataset= pd.read_csv('mnist_test.csv')
nrow = 28
ncol = 28


In [75]:
# from itertools import combinations
# l =range(10)
# combination=list(combinations(l, 2))
# # print (list(combinations(l, 2)))

# for (a,b) in combination:
#     i=a
#     j=b
i=0
j=1
print('Test between',i,'and',j)
def left_top(x):
    nsam = x.shape[0]
    nrow = x.shape[1]
    ncol = x.shape[2]
    z = np.zeros((nsam, nrow, ncol), dtype=np.double)
    data1 = np.concatenate((x, z), axis=1)
    data2 = np.concatenate((z, z), axis=1)
    data3 = np.concatenate((data1, data2), 2)
    return data3
def right_bottom(x):
    nsam = x.shape[0]
    nrow = x.shape[1]
    ncol = x.shape[2]
    z = np.zeros((nsam, nrow, ncol), dtype=np.double)
    data1 = np.concatenate((z, z), axis=1)
    data2 = np.concatenate((z, x), axis=1)
    data3 = np.concatenate((data1, data2), 2)
    return data3

def getType(dataset,x):
    index = np.where(dataset[:, 0] == x)
    for i in index:
        Type = dataset[i]
    return Type

def getPixel(x):
    pixel = x[:, 1:]
    return pixel
def getLabel(x):
    label = x[:, 0]
    return label

def traindata(dataset,i,j):
    dataset=dataset.values
#number i and j set
    type1=getType(dataset,i)
    type2=getType(dataset,j)
# data and label of the two sets
    x0 = getPixel(type1)
    x1 = getPixel(type2)
    y0 = getLabel(type1)
    y1 = getLabel(type2)
#label norm to adjust the pytorch cnn model
#     y0=y0-i
#     y1=y1-j+1
#  reshape X to 28*28
    nx1 = x1.shape[0]
    nx0 = x0.shape[0]
    x1_reshape = x1.reshape((nx1, nrow, ncol))
    x0_reshape = x0.reshape((nx0, nrow, ncol))
#Image transform------------------------
    left0 = left_top(x0_reshape)
    right1 = right_bottom(x1_reshape)
#concatenate the two type data-------------
    target = np.concatenate((left0, right1), axis=0)
    target= torch.from_numpy(target)
    target=torch.unsqueeze(target, 1)
    label = np.concatenate((y0, y1), axis=0)
    label=torch.from_numpy(label)
    return target,label

def testdata(dataset,i,j):
    dataset=dataset.values
#number i and j set
    type1=getType(dataset,i)
    type2=getType(dataset,j)
# data and label of the two sets
    x0 = getPixel(type1)
    x1 = getPixel(type2)
    y0 = getLabel(type1)
    y1 = getLabel(type2)
#label norm to adjust the pytorch cnn model
#     y0=y0-i
#     y1=y1-j+1
#  reshape X to 28*28
    nx1 = x1.shape[0]
    nx0 = x0.shape[0]
    x1_reshape = x1.reshape((nx1, nrow, ncol))
    x0_reshape = x0.reshape((nx0, nrow, ncol))
#Image transform------------------------
    right0 = right_bottom(x0_reshape)
    left1 = left_top(x1_reshape)
#concatenate the two type data-------------
    target = np.concatenate((right0, left1), axis=0)
    target= torch.from_numpy(target)
    target=torch.unsqueeze(target, 1)
    label = np.concatenate((y0, y1), axis=0)
    label=torch.from_numpy(label)
#     return target,label,right0,left1
    return target,label
(train_target,train_label)=traindata(train_dataset,i,j)
(test_target,test_label)=testdata(test_dataset,i,j)


# (train_target,train_label,xxxx,yyyy)=traindata(train_dataset,i,j)
# (test_target,test_label,xxx,yyy)=testdata(test_dataset,i,j)

Test between 0 and 1


In [76]:
# def plt_digit(x):
#     nrow = 56
#     ncol = 56
#     xsq = x.reshape((nrow,ncol))
#     plt.imshow(xsq,  cmap='Greys_r')
#     plt.xticks([])
#     plt.yticks([])

# plt_digit(train_target[1000,:])

# nplt=4
# for i in range(nplt):
#     plt.subplot(1,4,i+1)
#     plt_digit(xxx[i,:])

In [77]:
class train(Dataset):
    def __init__(self):
        self.len = train_label.shape[0]
        self.x = train_target
        self.y =train_label
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len
dataset1= train()
train_loader = DataLoader(dataset=dataset1,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

class test(Dataset):
    def __init__(self):

        self.len = test_label.shape[0]
        self.x = test_target
        self.y =test_label
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len
dataset2 = test()
test_loader = DataLoader(dataset=dataset2,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(2880,300)
        self.fc2 = nn.Linear(300,2)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = F.relu(out)   
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

model = ConvNet()
model = model.to(DEVICE)
model= model.double()
optimizer = optim.Adam(model.parameters())
# criterion = nn.CrossEntropyLoss()

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output=model(data)

        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data.double())

#             print(output)
#             print(target)

            test_loss += F.cross_entropy(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [3168/12665 (25%)]	Loss: 0.000000
Train Epoch: 1 [6368/12665 (50%)]	Loss: 0.000000
Train Epoch: 1 [9568/12665 (76%)]	Loss: 0.000000

Test set: Average loss: 0.0000, Accuracy: 2115/2115 (100.0000%)

Train Epoch: 2 [3168/12665 (25%)]	Loss: 0.000000
Train Epoch: 2 [6368/12665 (50%)]	Loss: 0.000000
Train Epoch: 2 [9568/12665 (76%)]	Loss: 0.000000

Test set: Average loss: 0.0000, Accuracy: 2115/2115 (100.0000%)

Train Epoch: 3 [3168/12665 (25%)]	Loss: 0.000000
Train Epoch: 3 [6368/12665 (50%)]	Loss: 0.000000
Train Epoch: 3 [9568/12665 (76%)]	Loss: 0.000000

Test set: Average loss: 0.0000, Accuracy: 2115/2115 (100.0000%)

Train Epoch: 4 [3168/12665 (25%)]	Loss: 0.000000
Train Epoch: 4 [6368/12665 (50%)]	Loss: 0.000000
Train Epoch: 4 [9568/12665 (76%)]	Loss: 0.000000

Test set: Average loss: 0.0000, Accuracy: 2115/2115 (100.0000%)

Train Epoch: 5 [3168/12665 (25%)]	Loss: 0.000000
Train Epoch: 5 [6368/12665 (50%)]	Loss: 0.000000
Train Epoch: 5 [9568/12665 (76%)]	Loss: 0.000000

